# This notebook computes the data for the other notebook to plot

## Sample type classification, ad hoc

In [1]:
def classify_organism(organism, taxonomy_hierarchy, assay_type):
    organism = organism.lower()
    assay_type = assay_type.lower()
    ranks = taxonomy_hierarchy
    
    #if 'environmental' in organism:
    #    return 'Environmental'
    if 'metagenome' in organism:
        return 'Metagenome'

    if ranks:
        if organism == 'severe acute respiratory syndrome coronavirus 2':
            return 'SARS-CoV-2'
        elif 'species' in ranks and ranks['species'] is not None and ranks['species'].lower() == 'homo sapiens':
            return 'Human'
        elif 'species' in ranks and ranks['species'] is not None and ranks['species'].lower() == 'mus musculus':
            return 'Mouse'
        elif 'superkingdom' in ranks and ranks['superkingdom'] is not None and ranks['superkingdom'] == 'Viruses':
            return 'Virome'
        elif 'class' in ranks and ranks['class'] is not None and ranks['class'] == 'Mammalia':
            return 'Mammal'
        elif 'phylum' in ranks and ranks['phylum'] is not None and ranks['phylum'] == 'Chordata':
            return 'Vertebrate'
        elif 'kingdom' in ranks and ranks['kingdom'] is not None and ranks['kingdom'] in ['Metazoa', 'Animalia']:
            return 'Invertebrate'
        elif 'kingdom' in ranks and ranks['kingdom'] is not None and ranks['kingdom'] == 'Fungi':
            return 'Fungus'
        elif 'kingdom' in ranks and ranks['kingdom'] is not None and ranks['kingdom'] in ['Viridiplantae', 'Plantae']:
            return 'Plant'
        elif 'superkingdom' in ranks and ranks['superkingdom'] is not None and ranks['superkingdom'] in ['Bacteria', 'Archaea']:
            return 'Prokaryote'

    #elif 'wgs' in assay_type or 'wga' in assay_type and 'mammal' in organism:
    #    return 'Mammal WGS'
    return 'Other'

def simplify_assay_type(assay_type):
    if assay_type == 'WGA' or assay_type == 'WGS':
        return "WGS/WGA"
    elif "RNA" in assay_type:
        return "RNA-Seq"
    return "OTHER"

## Parses local NCBI taxonomy

In [2]:
import csv

# Load names.dmp and nodes.dmp
def load_taxonomy_data():
    names = {}
    nodes = {}

    # Read names.dmp
    with open("taxonomy/names.dmp", "r") as f:
        reader = csv.reader(f, delimiter="|", quoting=csv.QUOTE_NONE)
        for row in reader:
            tax_id = row[0].strip()
            name = row[1].strip()
            name_class = row[3].strip()
            if name_class == "scientific name":
                names[tax_id] = name

    # Read nodes.dmp
    with open("taxonomy/nodes.dmp", "r") as f:
        reader = csv.reader(f, delimiter="|", quoting=csv.QUOTE_NONE)
        for row in reader:
            tax_id = row[0].strip()
            parent_tax_id = row[1].strip()
            rank = row[2].strip()
            nodes[tax_id] = {"parent_tax_id": parent_tax_id, "rank": rank}

    return names, nodes

# Function to get taxonomy hierarchy
def get_taxonomy_hierarchy(species_name, names, nodes):
    tax_id = None
    for k, v in names.items():
        if v.lower() == species_name.lower():
            tax_id = k
            break
    if tax_id is None:
        return f"Species '{species_name}' not found in the database"

    hierarchy = {"kingdom": None, "phylum": None, "class": None, "superkingdom": None, "species": None}
    current_id = tax_id

    while current_id != "1":  # 1 is the tax_id for the root
        if current_id not in nodes:
            break
        node = nodes[current_id]
        rank = node["rank"]
        if rank in hierarchy:
            hierarchy[rank] = names[current_id]
        current_id = node["parent_tax_id"]

    return hierarchy

def get_taxonomy_hierarchy_from_taxid(tax_id, names, nodes):
    hierarchy = {"kingdom": None, "phylum": None, "class": None, "superkingdom": None, "species": None}
    current_id = tax_id

    while current_id != "1":  # 1 is the tax_id for the root
        if current_id not in nodes:
            break
        node = nodes[current_id]
        rank = node["rank"]
        if rank in hierarchy:
            hierarchy[rank] = names[current_id]
        current_id = node["parent_tax_id"]

    return hierarchy

# Example usage
names, nodes = load_taxonomy_data()
species = "Homo sapiens"
taxonomy_hierarchy = get_taxonomy_hierarchy(species, names, nodes)
print(taxonomy_hierarchy)


{'kingdom': 'Metazoa', 'phylum': 'Chordata', 'class': 'Mammalia', 'superkingdom': 'Eukaryota', 'species': 'Homo sapiens'}


## Counts SRA accessions

In [3]:
import csv, io
import zstandard as zstd

from collections import Counter

c = Counter()

# File path to the compressed CSV file
file_path = 'sra_taxid.csv.zst'
counter = 0
counter_bad = 0
# Open and decompress the zstd file
with open(file_path, 'rb') as compressed_file:
    dctx = zstd.ZstdDecompressor()
    with dctx.stream_reader(compressed_file) as reader:
        # Wrap the decompressed stream with a text IO wrapper to read as CSV
        text_stream = io.TextIOWrapper(reader, encoding='utf-8')
        csv_reader = csv.reader(text_stream)
        
        # Process the CSV file line by line
        for row in csv_reader:
            try:
                acc, assay_type, organism, tax_id, taxonomic_rank, scientific_name = row
            except:
                #print("bad row",row)
                counter_bad += 1
                continue
            taxonomy_hierarchy = get_taxonomy_hierarchy_from_taxid(tax_id, names, nodes)
            org_class = classify_organism(organism,taxonomy_hierarchy,assay_type)
            simplified_assay_type = simplify_assay_type(assay_type)
            debug = False
            c[(org_class,simplified_assay_type)] += 1
            if debug:
                print(acc, assay_type, organism, tax_id, taxonomic_rank, scientific_name,sep="-----")
                print(org_class)
                print(taxonomy_hierarchy)
                print(simplified_assay_type)
            counter += 1
            #if counter==100000:
            #    break
print(counter,"good rows",counter_bad,"bad rows")
print(c)

81633149 good rows 1 bad rows
Counter({('Prokaryote', 'OTHER'): 24532110, ('Other', 'OTHER'): 12801884, ('SARS-CoV-2', 'OTHER'): 6475919, ('Metagenome', 'OTHER'): 5631474, ('Fungus', 'OTHER'): 5492637, ('Invertebrate', 'OTHER'): 4285349, ('Prokaryote', 'WGS/WGA'): 2881493, ('Human', 'OTHER'): 2793657, ('Plant', 'OTHER'): 2671713, ('Human', 'RNA-Seq'): 2127600, ('Mouse', 'RNA-Seq'): 2021849, ('Vertebrate', 'OTHER'): 1048542, ('Mouse', 'OTHER'): 997542, ('Metagenome', 'WGS/WGA'): 846229, ('Human', 'WGS/WGA'): 827534, ('Plant', 'RNA-Seq'): 618003, ('Prokaryote', 'RNA-Seq'): 606088, ('Plant', 'WGS/WGA'): 531197, ('Mammal', 'OTHER'): 458029, ('Other', 'WGS/WGA'): 428321, ('Invertebrate', 'RNA-Seq'): 418787, ('Virome', 'OTHER'): 414660, ('Other', 'RNA-Seq'): 331542, ('Mammal', 'RNA-Seq'): 330294, ('Invertebrate', 'WGS/WGA'): 311189, ('SARS-CoV-2', 'WGS/WGA'): 285678, ('Vertebrate', 'RNA-Seq'): 244459, ('Fungus', 'WGS/WGA'): 240797, ('Fungus', 'RNA-Seq'): 214536, ('Vertebrate', 'WGS/WGA'): 20

## Merge-sort-like streaming mechanism

In [4]:
import zstandard as zstd
import csv
import io

def read_zstd_file_line_by_line(file_path):
    with open(file_path, 'rb') as f:
        dctx = zstd.ZstdDecompressor()
        with dctx.stream_reader(f) as reader:
            text_stream = io.TextIOWrapper(reader, encoding='utf-8')
            csv_reader = csv.reader(text_stream)
            for row in csv_reader:
                yield row


def merge_sort_and_process(file1_path, file2_path, processing_func):
    file1_gen = read_zstd_file_line_by_line(file1_path)
    file2_gen = read_zstd_file_line_by_line(file2_path)

    file1_line = next(file1_gen, None)
    file2_line = next(file2_gen, None)

    while file1_line is not None and file2_line is not None:
        accession1 = file1_line[0]
        accession2 = file2_line[0]
        #break
        if accession1 < accession2:
            file1_line = next(file1_gen, None)
        elif accession1 > accession2:
            file2_line = next(file2_gen, None)
        else:
            processing_func(file1_line, file2_line)
            file1_line = next(file1_gen, None)
            file2_line = next(file2_gen, None)


## Now stream both the sra_taxid file and the diamond results file line by line and process accessions streamingly

In [6]:

from collections import defaultdict

# Initialize counters
palmcores_counter = defaultdict(int)
beetle_counter = defaultdict(int)
var_obelisk_counter = defaultdict(int)
var_Deltavirus_counter = defaultdict(int)
var_Osiris_counter = defaultdict(int)
papilloma_counter = defaultdict(int)

def process_lines(file1_line, file2_line):
    global palmcores_counter, beetle_counter, var_obelisk_counter, var_Deltavirus_counter, var_Osiris_counter, papilloma_counter
    #print("file1",file1_line)
    try:
        acc, palmcores, beetle, var_obelisk, var_Deltavirus, var_Osiris, papilloma = file1_line
    except:
        print("bad row file1", file1_line)
        return
    #print("file2",file2_line)
    try:
        acc2, assay_type, organism, tax_id, taxonomic_rank, scientific_name = file2_line
    except:
        print("bad row file2", file1_line)
        return
    assert(acc == acc2)
    taxonomy_hierarchy = get_taxonomy_hierarchy_from_taxid(tax_id, names, nodes)
    org_class = classify_organism(organism,taxonomy_hierarchy,assay_type)
    simplified_assay_type = simplify_assay_type(assay_type)
    
    # count all hits
    increment  = lambda x: int(x)
    # count just accessions
    increment  = lambda x: 1 if int(x) > 0 else 0
    
    # Increment counters
    key = (org_class, simplified_assay_type)
    palmcores_counter[key] += increment(palmcores)
    beetle_counter[key] += increment(beetle)
    var_obelisk_counter[key] += increment(var_obelisk)
    var_Deltavirus_counter[key] += increment(var_Deltavirus)
    var_Osiris_counter[key] += increment(var_Osiris)
    papilloma_counter[key] += increment(papilloma)

# Paths to the files
file1_path = '../results/output.sorted.txt.zst'
file2_path = '../plot/sra_taxid.csv.zst'

# Run the merge sort and process function
merge_sort_and_process(file1_path, file2_path, process_lines)

print("palmcores_counter = ", dict(palmcores_counter))
print("beetle_counter =", dict(beetle_counter))
print("Var_Obelisk_counter = ", dict(var_obelisk_counter))
print("Var_Deltavirus_counter = ", dict(var_Deltavirus_counter))
print("Var_Osiris_counter =", dict(var_Osiris_counter))
print("Papilloma_counter =", dict(papilloma_counter))
    

bad row file2 ['SRR19732797', '126', '926', '0', '0', '0', '5']
palmcores_counter =  {('Prokaryote', 'WGS/WGA'): 2341160, ('Human', 'OTHER'): 1529268, ('Metagenome', 'OTHER'): 2628747, ('Plant', 'WGS/WGA'): 407245, ('Mouse', 'OTHER'): 799481, ('Prokaryote', 'OTHER'): 122250, ('Vertebrate', 'OTHER'): 227638, ('Plant', 'OTHER'): 567399, ('Mammal', 'WGS/WGA'): 135832, ('Mouse', 'WGS/WGA'): 51059, ('Other', 'WGS/WGA'): 174413, ('Human', 'WGS/WGA'): 360410, ('Vertebrate', 'WGS/WGA'): 145488, ('Invertebrate', 'OTHER'): 188346, ('Metagenome', 'WGS/WGA'): 615214, ('Plant', 'RNA-Seq'): 503686, ('Vertebrate', 'RNA-Seq'): 195756, ('Virome', 'WGS/WGA'): 53495, ('Human', 'RNA-Seq'): 1487554, ('Virome', 'OTHER'): 47044, ('Mammal', 'OTHER'): 215172, ('Fungus', 'OTHER'): 67267, ('Invertebrate', 'RNA-Seq'): 294334, ('Other', 'OTHER'): 125204, ('Invertebrate', 'WGS/WGA'): 218660, ('Mammal', 'RNA-Seq'): 252907, ('Other', 'RNA-Seq'): 70342, ('Mouse', 'RNA-Seq'): 1693343, ('Metagenome', 'RNA-Seq'): 100862,

## Get petabases per type

In [80]:

from collections import defaultdict

# Initialize counters
petabases_counter = defaultdict(int)
nbacc_counter = defaultdict(int)

def process_lines_Athenastats(file1_line, file2_line):
    global petabases_counter, nbacc_counter
    #print("file1",file1_line)
    try:
        acc,mbases,mbytes,avgspotlen,librarylayout,instrument = file1_line
    except:
        print("bad row file1", file1_line)
        return
    #print("file2",file2_line)
    try:
        acc2, assay_type, organism, tax_id, taxonomic_rank, scientific_name = file2_line
    except:
        print("bad row file2", file1_line)
        return
    assert(acc == acc2)
    taxonomy_hierarchy = get_taxonomy_hierarchy_from_taxid(tax_id, names, nodes)
    org_class = classify_organism(organism,taxonomy_hierarchy,assay_type)
    simplified_assay_type = simplify_assay_type(assay_type)
    
    # Increment counters
    key = (org_class, simplified_assay_type)
    petabases_counter[key] += int(mbases)
    nbacc_counter[key] += 1

# Paths to the files
file1_path = '/home/ec2-user/erc-unitigs-prod/Athena_Dec_10_public.sorted.csv.zst'
file2_path = '../plot/sra_taxid.csv.zst'

# Run the merge sort and process function
merge_sort_and_process(file1_path, file2_path, process_lines_Athenastats)

print("petabases_counter = ", dict(petabases_counter))
print("nbacc_counter =", dict(nbacc_counter))


bad row file1 ['SRR19732797', '1414', '87', '258', 'PAIRED', '\nSRR19732798"', '691', '273', '281', 'PAIRED', 'NextSeq 550']
petabases_counter =  {('Prokaryote', 'WGS/WGA'): 1337536637, ('Human', 'OTHER'): 4175010458, ('Metagenome', 'OTHER'): 698673154, ('Plant', 'WGS/WGA'): 3948094067, ('Mouse', 'OTHER'): 2099232371, ('Prokaryote', 'OTHER'): 92153348, ('Mammal', 'OTHER'): 696688033, ('Vertebrate', 'OTHER'): 452695920, ('Plant', 'OTHER'): 1293976973, ('Mammal', 'WGS/WGA'): 3108752753, ('Invertebrate', 'OTHER'): 487270872, ('Mouse', 'WGS/WGA'): 741610048, ('Other', 'WGS/WGA'): 372653970, ('Human', 'WGS/WGA'): 3437232410, ('Vertebrate', 'WGS/WGA'): 1403041500, ('Metagenome', 'WGS/WGA'): 2695259386, ('Plant', 'RNA-Seq'): 2580159244, ('Vertebrate', 'RNA-Seq'): 883797364, ('Virome', 'WGS/WGA'): 25321925, ('Human', 'RNA-Seq'): 6386799101, ('Metagenome', 'RNA-Seq'): 361678065, ('Virome', 'OTHER'): 25097506, ('Fungus', 'OTHER'): 126223433, ('Invertebrate', 'RNA-Seq'): 1112706460, ('Other', 'OT

## anello/papilloma 

In [7]:

from collections import defaultdict

# Initialize counters
papilloma_counter = defaultdict(int)
anello_counter = defaultdict(int)

def process_lines(file1_line, file2_line):
    global palmcores_counter, anello_counter
    #print("file1",file1_line)
    try:
        acc, anello, papilloma = file1_line
    except:
        print("bad row file1", file1_line)
        return
    #print("file2",file2_line)
    try:
        acc2, assay_type, organism, tax_id, taxonomic_rank, scientific_name = file2_line
    except:
        print("bad row file2", file1_line)
        return
    assert(acc == acc2)
    taxonomy_hierarchy = get_taxonomy_hierarchy_from_taxid(tax_id, names, nodes)
    org_class = classify_organism(organism,taxonomy_hierarchy,assay_type)
    simplified_assay_type = simplify_assay_type(assay_type)
    
    # count all hits
    increment  = lambda x: int(x)
    # count just accessions
    increment  = lambda x: 1 if int(x) > 0 else 0
    
    # Increment counters
    key = (org_class, simplified_assay_type)
    papilloma_counter[key] += increment(papilloma)
    anello_counter[key] += increment(anello)

# Paths to the files
file1_path = '/home/ec2-user/logan-analysis/analyses/july1-diamond/results/count_accessions2.res.sorted.txt.zst'
file2_path = '../plot/sra_taxid.csv.zst'

# Run the merge sort and process function
merge_sort_and_process(file1_path, file2_path, process_lines)


print("Papilloma_counter =", dict(papilloma_counter))
print("Anello_counter =", dict(anello_counter))
    

bad row file2 ['SRR19732797', '1', '1']
Papilloma_counter = {('Human', 'OTHER'): 710150, ('Metagenome', 'OTHER'): 186291, ('Plant', 'WGS/WGA'): 267894, ('Mouse', 'OTHER'): 472192, ('Prokaryote', 'WGS/WGA'): 1166860, ('Plant', 'OTHER'): 249587, ('Mammal', 'WGS/WGA'): 51204, ('Mouse', 'WGS/WGA'): 22259, ('Other', 'WGS/WGA'): 76732, ('Human', 'WGS/WGA'): 77781, ('Vertebrate', 'WGS/WGA'): 71462, ('Metagenome', 'WGS/WGA'): 449894, ('Plant', 'RNA-Seq'): 435176, ('Vertebrate', 'OTHER'): 105502, ('Virome', 'WGS/WGA'): 9242, ('Human', 'RNA-Seq'): 961448, ('Virome', 'OTHER'): 6755, ('Invertebrate', 'OTHER'): 87059, ('Invertebrate', 'RNA-Seq'): 200264, ('Invertebrate', 'WGS/WGA'): 179043, ('Mammal', 'RNA-Seq'): 204909, ('Fungus', 'OTHER'): 24989, ('Mammal', 'OTHER'): 74716, ('Other', 'RNA-Seq'): 47721, ('Mouse', 'RNA-Seq'): 1075289, ('Prokaryote', 'OTHER'): 26917, ('Vertebrate', 'RNA-Seq'): 149773, ('Metagenome', 'RNA-Seq'): 43240, ('Fungus', 'RNA-Seq'): 64637, ('Fungus', 'WGS/WGA'): 64902, ('Vir